In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import cv2
import math
import numpy as np
from skimage import measure
from skimage.measure import regionprops
from skimage.morphology import opening, disk
from scipy.stats import mode
import matplotlib.pyplot as plt

In [0]:
PATIENT_ID = ['221','229','230','232','239','242','246','252','263']
proliferationIndex = ["LOW", "INTERMEDIATE", "HIGH"]
ROOT_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/Ki67/LadderNet/"

In [4]:
PATH = []

for patient in (os.listdir(ROOT_PATH + 'Train/')):
  PATH.append(ROOT_PATH + "Train/" + patient)

for patient in (os.listdir(ROOT_PATH + 'Test/')):
  PATH.append(ROOT_PATH + "Test/" + patient)

print(len(PATH))

9


In [0]:
def getCountUtil(image):

  # Opening operation
  selem = disk(15)
  opened = opening(image, selem)

  # Finding the regions/nuclei
  nuclei = measure.label(opened)
  props = regionprops(nuclei)

  # Removing small pixel patches
  if(len(props)==0):
    return 0
  new_props = list.copy(props)
  for i in range(len(props)):
    if props[i].area <= 200:
      new_props.pop(i)
  props = new_props

  # Caclulating average nuclei size
  if(len(props)==0):
    return 0
  avg_size = 0
  for i in range(len(props)):
    avg_size += props[i].area 
  avg_size /= len(props)

  # Splitting Overlapping Nuclei and getting count
  count = len(props)
  for i in range(len(props)):
    if(props[i].area > avg_size):
      count += math.floor(props[i].area/avg_size - 1)  
  
  return count

In [0]:
def getCount(image,intensity):
  new_image = np.copy(image)
  new_image[new_image!=intensity] = 0
  count = getCountUtil(new_image)
  return count

In [0]:
def getProliferationIndex(IPCount,INCount):
  PI = IPCount/(IPCount + INCount)*100
  return PI

In [0]:
def getResult(score):
  result = 0
  if(score<=15):
    result = 0      # Low Proliferation of Brest Tumour
  elif(score<=30):
    result = 1      # Intermediate Proliferation of Breast Tumour
  else:
    result = 2      # High Proliferation of Breast Tumour
  return result

In [0]:
def ScoreCalculation(mask):
  
  immunopositive = getCount(mask,255)
  immunonegative = getCount(mask,128)

  print("Immunopositive : " , immunopositive)
  print("Immunonegative : " , immunonegative)

  PI = getProliferationIndex(immunopositive,immunonegative)
  result = getResult(PI)

  print("Score : " , PI)
  print("Proliferation Rate : ", proliferationIndex[result] , "\n")

  return (PI,result)

In [0]:
# Patient-wise result
def patientWiseResultUtil(maskPath):
  mask = np.load(maskPath)
  result = np.zeros(len(mask), dtype=int)
  score = np.zeros(len(mask))
  for i in range(len(mask)):
    score[i],result[i] = ScoreCalculation(mask[i])
  return (mode(result).mode)[0]

In [0]:
def patientWiseResult(PatientID):

  path = ""
  for p in PATH:
     num = (p.split('/')[-1]).split('.')[0]
     if(num==PatientID):
       path = p
       break

  print("PATIENT ", PatientID,"\n")
  res = patientWiseResultUtil(path)
  
  print("IHC ",PatientID," Proliferation rate : ", proliferationIndex[res])
  print("-----------------------------------------------------------------")

In [19]:
for id in PATIENT_ID:
  patientWiseResult(id)

PATIENT  221 

Immunopositive :  34
Immunonegative :  126
Score :  21.25
Proliferation Rate :  INTERMEDIATE 

Immunopositive :  32
Immunonegative :  142
Score :  18.39080459770115
Proliferation Rate :  INTERMEDIATE 

Immunopositive :  27
Immunonegative :  120
Score :  18.367346938775512
Proliferation Rate :  INTERMEDIATE 

Immunopositive :  37
Immunonegative :  118
Score :  23.870967741935484
Proliferation Rate :  INTERMEDIATE 

Immunopositive :  30
Immunonegative :  136
Score :  18.072289156626507
Proliferation Rate :  INTERMEDIATE 

Immunopositive :  40
Immunonegative :  128
Score :  23.809523809523807
Proliferation Rate :  INTERMEDIATE 

Immunopositive :  41
Immunonegative :  123
Score :  25.0
Proliferation Rate :  INTERMEDIATE 

Immunopositive :  26
Immunonegative :  127
Score :  16.99346405228758
Proliferation Rate :  INTERMEDIATE 

Immunopositive :  44
Immunonegative :  136
Score :  24.444444444444443
Proliferation Rate :  INTERMEDIATE 

Immunopositive :  38
Immunonegative :  113